In [63]:
import pandas as pd 
import numpy as np 
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.tree import export_graphviz 
from sklearn.model_selection import KFold
from pydot import pydot
#cloned into https://github.com/erocarrera/pydot
from pydot import dot_parser
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import norm

In [2]:
food = pd.read_csv("food_full.csv")
food = food.drop(["Unnamed: 0", "date"], axis = 1)
company = pd.read_csv("company_full.csv")
company = company.drop("company", axis = 1)


### Looking at whether a company would be a one time or multiple time donations before first donation

In [3]:
#features = food.drop(["Unnamed: 0", "Id", "date", "number", "street", "city", "company", "zip", "lat", "lon", "pickup", "planned", "day", "year"], axis = 1)
#features.loc[(features["state"] != "California") & (features["state"] != "New York"), "state"] = "underdeveloped"
features = company.drop(["one_donation"], axis = 1) 

In [4]:
features.shape

(242, 20)

In [5]:
y = company["one_donation"]
#features = pd.get_dummies(features)
feature_list = list(features.columns)
features = np.array(features)

In [6]:
train_features, test_features, train_labels, test_labels = train_test_split(features, y, test_size = 0.25, random_state = 23156)

In [7]:
rf = RandomForestClassifier(n_estimators = 1000, random_state = 2756)
rf.fit(train_features, train_labels);

In [8]:
predictions = rf.predict(test_features)
predictions

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0], dtype=int64)

In [9]:
sum(test_labels == predictions)/len(test_labels)

0.7213114754098361

In [49]:
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: size_50.0            Importance: 0.1
Variable: size_200.0           Importance: 0.08
Variable: size_5000.0          Importance: 0.07
Variable: state_California     Importance: 0.06
Variable: state_New York       Importance: 0.06
Variable: state_underdeveloped Importance: 0.06
Variable: type_comm            Importance: 0.06
Variable: type_bus             Importance: 0.05
Variable: type_food            Importance: 0.05
Variable: size_10.0            Importance: 0.05
Variable: size_1000.0          Importance: 0.05
Variable: type_applied         Importance: 0.04
Variable: type_edu-health      Importance: 0.04
Variable: type_sale            Importance: 0.04
Variable: type_web             Importance: 0.04
Variable: type_fin             Importance: 0.03
Variable: type_other           Importance: 0.03
Variable: type_soft            Importance: 0.03
Variable: size_10001.0         Importance: 0.03
Variable: size_10000.0         Importance: 0.03


In [158]:
tree = rf.estimators_[5]
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree.dot')
graph.write_png('tree.png')

NameError: name 'dot_parser' is not defined

In [159]:
dot_parser.parse_dot_data("tree.dot")

TypeError: must be str, not ParseException

In [52]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ["auto", "sqrt"]
max_depth = [int(x) for x in np.linspace(start = 3, stop = 60, num = 20)] 
#max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 10, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 5, num = 5)]
bootstrap = [True, False] 

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [59]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                              n_iter = 100, cv = 10, verbose = 2, random_state = 6819,
                              n_jobs = -1)
rf_random.fit(train_features, train_labels)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:  9.7min finished


RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60], 'min_samples_split': [2, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_leaf': [1, 2, 3, 4, 5], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=6819, refit=True,
          return_train_score='warn

In [54]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 3,
 'max_features': 'auto',
 'min_samples_leaf': 3,
 'min_samples_split': 5,
 'n_estimators': 1800}

In [60]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 3,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 7,
 'n_estimators': 800}

In [7]:
rf = RandomForestClassifier(n_estimators = 800, 
                            bootstrap = True,
                            max_depth = 3,
                            max_features = "sqrt",
                            min_samples_leaf = 5,
                            min_samples_split = 7,
                            random_state = 2332)
rf.fit(train_features, train_labels);

In [8]:
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: state_underdeveloped Importance: 0.19
Variable: size_1000.0          Importance: 0.16
Variable: size_50.0            Importance: 0.07
Variable: state_California     Importance: 0.06
Variable: state_New York       Importance: 0.06
Variable: type_applied         Importance: 0.06
Variable: type_comm            Importance: 0.06
Variable: type_edu-health      Importance: 0.05
Variable: type_bus             Importance: 0.04
Variable: type_sale            Importance: 0.04
Variable: type_fin             Importance: 0.03
Variable: type_food            Importance: 0.03
Variable: size_5000.0          Importance: 0.03
Variable: size_200.0           Importance: 0.03
Variable: type_other           Importance: 0.02
Variable: type_soft            Importance: 0.02
Variable: type_web             Importance: 0.02
Variable: size_10.0            Importance: 0.02
Variable: size_10001.0         Importance: 0.02
Variable: size_10000.0         Importance: 0.0


In [9]:
sum(rf.predict(test_features) == test_labels)/len(test_labels)

0.7213114754098361

In [14]:
from sklearn.metrics import precision_score, recall_score
precision_accuracy = precision_score(test_labels, rf.predict(test_features))
recall_accuracy = recall_score(test_labels, rf.predict(test_features))

print('Precision Score: {0:0.2f}'.format(
      precision_accuracy))
print('Recall Score: {0:0.2f}'.format(
      recall_accuracy))

Precision Score: 0.67
Recall Score: 0.21


In [11]:
preds = rf.predict(test_features)
predicted_one_time = [i for i in range(len(preds)) if preds[i] == 1]
predicted_multi_time = [i for i in range(len(preds)) if preds[i] == 0]
true_one_time = [i for i in range(len(test_labels)) if np.array(test_labels)[i] == 1]
true_multi_time = [i for i in range(len(test_labels)) if np.array(test_labels)[i] == 0]
true_positives = len(np.intersect1d(predicted_one_time, true_one_time))
false_positives = len(np.intersect1d(predicted_one_time, true_multi_time))
true_negatives = len(np.intersect1d(predicted_multi_time, true_multi_time))
false_negatives = len(np.intersect1d(predicted_multi_time, true_one_time))

In [16]:
precision = true_positives/(true_positives + false_positives)
recall = true_positives/(true_positives + false_negatives)

In [17]:
print(precision, recall)

0.6666666666666666 0.21052631578947367


In [24]:
false_negatives

15

A lot of false negatives: assume more companies are multi-time donors when they are in fact one time donors. 

In [59]:
sum(test_labels == 0)/len(test_labels)

0.6885245901639344

### Testing which company types can be combined

In [39]:
food["type"].unique()

array(['food', 'other', 'comm', 'fin', 'soft', 'bus', 'edu-health',
       'applied', 'web', 'sale'], dtype=object)

In [47]:
company_food = company[company["type_food"] == 1]["one_donation"]
company_other = company[company["type_other"] == 1]["one_donation"]
company_comm = company[company["type_comm"] == 1]["one_donation"]
company_fin = company[company["type_fin"] == 1]["one_donation"]
company_soft = company[company["type_soft"] == 1]["one_donation"]
company_bus = company[company["type_bus"] == 1]["one_donation"]
company_eh = company[company["type_edu-health"] == 1]["one_donation"]
company_applied = company[company["type_applied"] == 1]["one_donation"]
company_web = company[company["type_web"] == 1]["one_donation"]
company_sale = company[company["type_sale"] == 1]["one_donation"]
dists_name = ["Food", "Other", "Communication", "Finance", "Software", 
             "Business", "Education-Health", "Applied", "Web", "Sale"]
dists = [company_food, company_other, company_comm, company_fin, company_soft,
        company_bus, company_eh, company_applied, company_web, company_sale]

In [51]:
props = []
for i in range(len(dists)):
    props.append(sum(dists[i] == 1)/len(dists[i]))
props = np.array(props)

In [52]:
props

array([ 0.37142857,  0.35294118,  0.42307692,  0.2962963 ,  0.1875    ,
        0.29032258,  0.42857143,  0.45454545,  0.42105263,  0.35294118])

In [94]:
norm.interval(.95)

(-1.959963984540054, 1.959963984540054)

In [92]:
print()
for i in range(1, len(dists)): 
    n1 = len(dists[0])
    n2 = len(dists[i])
    p1 = props[0]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Food and " + dists_name[i] + ': {0:0.2f}'.format(
      z))


Food and Other: 0.13
Food and Communication: -0.41
Food and Finance: 0.62
Food and Software: 1.31
Food and Business: 0.70
Food and Education-Health: -0.42
Food and Applied: -0.70
Food and Web: -0.36
Food and Sale: 0.13


In [97]:
for i in range(2, len(dists)): 
    n1 = len(dists[1])
    n2 = len(dists[i])
    p1 = props[1]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Other and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Other and Communication: -0.46
Other and Finance: 0.39
Other and Software: 1.07
Other and Business: 0.45
Other and Education-Health: -0.47
Other and Applied: -0.69
Other and Web: -0.42
Other and Sale: 0.00


In [98]:
for i in range(3, len(dists)): 
    n1 = len(dists[2])
    n2 = len(dists[i])
    p1 = props[2]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Communication and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Communication and Finance: 0.96
Communication and Software: 1.57
Communication and Business: 1.05
Communication and Education-Health: -0.04
Communication and Applied: -0.24
Communication and Web: 0.01
Communication and Sale: 0.46


In [99]:
for i in range(4, len(dists)): 
    n1 = len(dists[3])
    n2 = len(dists[i])
    p1 = props[3]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Finance and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Finance and Software: 0.79
Finance and Business: 0.05
Finance and Education-Health: -0.95
Finance and Applied: -1.25
Finance and Web: -0.87
Finance and Sale: -0.39


In [100]:
for i in range(5, len(dists)): 
    n1 = len(dists[4])
    n2 = len(dists[i])
    p1 = props[4]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Software and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Software and Business: -0.77
Software and Education-Health: -1.55
Software and Applied: -1.82
Software and Web: -1.48
Software and Sale: -1.07


In [101]:
for i in range(6, len(dists)): 
    n1 = len(dists[5])
    n2 = len(dists[i])
    p1 = props[5]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Business and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Business and Education-Health: -1.03
Business and Applied: -1.36
Business and Web: -0.95
Business and Sale: -0.45


In [102]:
for i in range(7, len(dists)): 
    n1 = len(dists[6])
    n2 = len(dists[i])
    p1 = props[6]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Education-Health and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Education-Health and Applied: -0.19
Education-Health and Web: 0.05
Education-Health and Sale: 0.47


In [103]:
for i in range(8, len(dists)): 
    n1 = len(dists[7])
    n2 = len(dists[i])
    p1 = props[7]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Applied and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Applied and Web: 0.23
Applied and Sale: 0.69


In [104]:
for i in range(9, len(dists)): 
    n1 = len(dists[8])
    n2 = len(dists[i])
    p1 = props[8]
    p2 = props[i]
    p = (n1 * p1 + n2 * p2)/(n1 + n2)
    z = (p1 - p2)/(p * (1 - p) * (1/n1 + 1/n2))**0.5
    print("Web and " + dists_name[i] + ': {0:0.2f}'.format(
      z))

Web and Sale: 0.42
